In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import f
import statsmodels.stats.stattools as stattools
from scipy.stats import mstats
from scipy.stats import zscore
from scipy.stats import shapiro


# Lendo a planilha com os dados 

In [2]:
def ler_planilha (path):
    df = pd.read_excel(path, sheet_name='Dados')
    df = df.set_index("Replicatas") #df.set_index("Replicatas", inplace=True)
    df = df.apply(pd.to_numeric, errors='coerce')# Converter as colunas para formato numérico, se necessário
    df = df.round(2)
    return df

In [3]:
df  =  ler_planilha ('Testes_Estatisticos.xlsx')

In [4]:
df

,Padrão 1,Padrão 2,Padrão 3,Padrão 4,Padrão 5,Padrão 6
Replicatas,,,,,,
1,73439.33,114422.95,137847.37,168777.89,219072.22,281650.94
2,74709.57,114855.20,136493.41,172028.96,222644.05,276065.01
3,75383.99,116134.25,135820.57,172089.50,222438.49,277585.04
4,76085.28,117401.73,136135.26,173948.81,223538.30,278630.83
5,76761.00,117268.61,137848.39,175236.13,220192.94,281322.57
6,76516.32,117733.62,138811.59,175135.40,220294.16,280352.98
7,76067.53,115657.58,138185.08,175373.97,220294.16,281291.21


In [5]:
#df.describe()

In [6]:
df  =  ler_planilha ('Testes_Estatisticos.xlsx')

# Calcular a média e desvio padrão de cada coluna
medias = df.mean().round(2)
desvios_padrao = df.std().round(2)
variancias = df.var().round(2)

#print(medias)
#print(desvios_padrao)
#print(variancias)


In [7]:
dfmin = df.min()
#dfmin

In [8]:
# Calcular os valores normalizados Gmin e Gmax
Gmin = ((medias - df.min()) / desvios_padrao).round(3)
Gmax = ((df.max() - medias) / desvios_padrao).round(3)
#Gmin
#Gmax

In [9]:
# Criar um DataFrame com as estatísticas calculadas
estatisticas_df = pd.DataFrame([medias, desvios_padrao, Gmin, Gmax, variancias], 
                               index=['Média', 'Stand. Dev.', 'Gmin', 'Gmax', 'Variância'])

# Adicionar o DataFrame de estatísticas ao final do DataFrame original
df = pd.concat([df, estatisticas_df])


#df

# Teste Shapiro Wilk

In [26]:
def shapiro_teste(col):
    stat, p = shapiro(col)
    if p > 0.05:
        return True
    else:
        return False

def shapiro_estatistica(col):
    stat, p = shapiro(col)
    return stat

def shapiro_p(col):
    stat, p = shapiro(col)
    return p

def shapiro_df(df):
    estatisticas_shapiro = df.apply(shapiro_estatistica)
    estatisticas_df = pd.DataFrame([estatisticas_shapiro], index=['Estatística W'])
    df = pd.concat([df, estatisticas_df])

    p_shapiro = df.apply(shapiro_p)
    p_df = pd.DataFrame([p_shapiro], index=['Valor p'])
    df = pd.concat([df, p_df])

    normal_shapiro = df.apply(shapiro_teste).apply(lambda x: "Sim" if x else "Não")
    normal_df = pd.DataFrame([normal_shapiro], index=['Distribuição Normal'])
    df = pd.concat([df, normal_df])
    return df

In [29]:
df  =  ler_planilha ('Testes_Estatisticos.xlsx')
resultados = shapiro_df(df)
resultados

,Padrão 1,Padrão 2,Padrão 3,Padrão 4,Padrão 5,Padrão 6
1,73439.33,114422.95,137847.37,168777.89,219072.22,281650.94
2,74709.57,114855.2,136493.41,172028.96,222644.05,276065.01
3,75383.99,116134.25,135820.57,172089.5,222438.49,277585.04
4,76085.28,117401.73,136135.26,173948.81,223538.3,278630.83
5,76761.0,117268.61,137848.39,175236.13,220192.94,281322.57
6,76516.32,117733.62,138811.59,175135.4,220294.16,280352.98
7,76067.53,115657.58,138185.08,175373.97,220294.16,281291.21
Estatística W,0.906179,0.918947,0.910628,0.859808,0.900818,0.886165
Valor p,0.000003,0.000002,0.000002,0.000003,0.000002,0.000002
Distribuição Normal,Não,Não,Não,Não,Não,Não


In [28]:
#df  =  ler_planilha ('Testes_Estatisticos.xlsx')
#from scipy.stats import shapiro
#for col in df.columns:
#    stat, p = shapiro(df[col])
#    print(f'Coluna {col}:')
#    print('Estatística de teste:', stat)
#    print('Valor p:', p)
#    if p > 0.05:
#        print('Os dados parecem seguir uma distribuição normal.\n')
#    else:
#        print('Os dados não seguem uma distribuição normal.\n')

IndentationError: unexpected indent (3334724584.py, line 10)

# Teste Grubbs

In [13]:
# Teste de outliers (Grubbs)
df  =  ler_planilha ('Testes_Estatisticos.xlsx')

def grubbs_test(data):
    mean = data.mean()
    std_dev = data.std()
    n = len(data)

    max_outlier = max(data)
    min_outlier = min(data)

    z = (max_outlier - mean) / std_dev if max_outlier > mean else (min_outlier - mean) / std_dev

    critical_value = stats.t.ppf(1 - 0.05 / (2 * n), n - 2)

    return z > critical_value

for col in df.columns:
    is_outlier = grubbs_test(df[col])
    print(f'Coluna {col}:')
    if is_outlier:
        print('Outlier encontrado.\n')
    else:
        print('Nenhum outlier encontrado.\n')

Coluna Padrão 1:
Nenhum outlier encontrado.

Coluna Padrão 2:
Nenhum outlier encontrado.

Coluna Padrão 3:
Nenhum outlier encontrado.

Coluna Padrão 4:
Nenhum outlier encontrado.

Coluna Padrão 5:
Nenhum outlier encontrado.

Coluna Padrão 6:
Nenhum outlier encontrado.



# Teste Cochran

In [51]:
df  =  ler_planilha ('Testes_Estatisticos.xlsx')
df_inicial = df.copy()

# Teste de homocedasticidade (Cochran)
#stat, pvalue = stats.levene(df['Padrão 1'], df['Padrão 2'], df['Padrão 3'], df['Padrão 4'], df['Padrão 5'], df['Padrão 6'])
#print(stat)
#if pvalue < 0.05:
    #print('Os dados não são homocedásticos.')
#else:
   #rint('Os dados são homocedásticos.')

# Calcula as variâncias de cada grupo
# O argumento opcional ddof indica o número de graus de liberdade a serem usados no cálculo da variância.
#Especificar ddof=1 significa que estamos calculando a variância amostral (usando N-1 como denominador, onde N é o número de observações).
#Isso é comumente usado em estatísticas para ajustar a variância amostral para um tamanho de amostra menor.
variances_df = df.var(ddof=1)

variances_df = pd.DataFrame([variances_df], index=['Variância'])
df = pd.concat([df, variances_df])

# Identifica a variância máxima e mínima
max_variance = variances.max()
variance_max_df= pd.DataFrame([max_variance], index=['Variância Max'])
df = pd.concat([df, variance_max_df])

min_variance = variances.min()
variance_min_df= pd.DataFrame([min_variance], index=['Variância Min'])
df = pd.concat([df, variance_min_df])

soma_variances = df.loc['Variância'].sum()
soma_variances_df= pd.DataFrame([soma_variances], index=['Soma das Variâncias'])
df = pd.concat([df, soma_variances_df])

# Calcula a estatística de teste Cochrane
sum_variances = variances.sum()
cochrane_statistic = max_variance / sum_variances # Ccalc =  Variância Max/Soma das Variâncias
#cochrane_statistic = max_variance / min_variance  # Ccalc =  Variância Max/Variância Min
cochrane_statistic_df= pd.DataFrame([cochrane_statistic], index=['Cochrane Estatistica'])
df = pd.concat([df, cochrane_statistic_df])

# Graus de liberdade para o numerador e denominador da distribuição F
df_num = df_inicial.shape[0] - 1  # Graus de liberdade do numerador (entre grupos)
df_den = df_inicial.size - df.shape[0]  # Graus de liberdade do denominador (dentro dos grupos)

# Valor crítico com 95% de confiança (alpha = 0.05)
valor_critico = f.ppf(1 - 0.05, df_num, df_den)
valor_critico_df= pd.DataFrame([valor_critico], index=['Cochrane Tabelado'])
df = pd.concat([df, valor_critico_df])

if cochrane_statistic < valor_critico:
    Resultado = "HOMOCEDÁSTICO"
    Resultado_df= pd.DataFrame([Resultado], index=['Resultado'])
    df = pd.concat([df, Resultado_df])
else:
    Resultado = "Não HOMOCEDÁSTICO"
    Resultado_df= pd.DataFrame([Resultado], index=['Resultado'])
    df = pd.concat([df, Resultado_df])

df = df.fillna("")

df



,Padrão 1,Padrão 2,Padrão 3,Padrão 4,Padrão 5,Padrão 6,0
1,73439.33,114422.95,137847.37,168777.89,219072.22,281650.94,
2,74709.57,114855.2,136493.41,172028.96,222644.05,276065.01,
3,75383.99,116134.25,135820.57,172089.5,222438.49,277585.04,
4,76085.28,117401.73,136135.26,173948.81,223538.3,278630.83,
5,76761.0,117268.61,137848.39,175236.13,220192.94,281322.57,
6,76516.32,117733.62,138811.59,175135.4,220294.16,280352.98,
7,76067.53,115657.58,138185.08,175373.97,220294.16,281291.21,
Variância,1356940.381162,1700371.31219,1310714.701424,5888811.54399,2711450.914957,4680550.313067,
Variância Max,,,,,,,5888811.54399
Variância Min,,,,,,,1310714.701424


# Teste Anova

In [15]:
df  =  ler_planilha ('Testes_Estatisticos.xlsx')

# ANOVA (falta de ajuste e erro puro)
F, pvalue = stats.f_oneway(df['Padrão 1'], df['Padrão 1'])
if pvalue < 0.05:
    print('Existe uma diferença significativa entre os grupos.')
else:
    print('Não existe uma diferença significativa entre os grupos.')

Não existe uma diferença significativa entre os grupos.


# Durbin Watson

In [16]:
df  =  ler_planilha ('Testes_Estatisticos.xlsx')

# Teste de independência dos resíduos (Durbin Watson)
# Obter o teste de Durbin-Watson
dw = stattools.durbin_watson(df['Padrão 1'])

# Imprimir o resultado do teste
print(dw)

8.199227618926422e-05
